
## Calibration tests
### Pose2Sim calibration.py functions
### KaggleHub dataset "danielwe14/stereocamera-chessboard-pictures"

Auteurs : Léa Drolet-Roy

Création : 2025-02-14

Dernière modification :

In [1]:
from pose2sim_trampo import calibration
import os

In [2]:
rcam = r'C:\Users\LEA\Desktop\Poly\Trampo\kaggledata\data\imgs\rightcamera'
lcam = r'C:\Users\LEA\Desktop\Poly\Trampo\kaggledata\data\imgs\leftcamera'

In [ ]:
intrinsics_config_dict = {'calculate_intrinsics':True, 'overwrite_intrinsics':True, 'show_detection_intrinsics':False, 'intrinsics_method':'board',
                          'intrinsics_extension': 'png', 'extract_every_N_sec':1, 'intrinsics_corners_nb':[11,7], 'intrinsics_square_size':30,
                          'board': {'intrinsics_extension': 'png', 'show_reprojection_error': True,
                                    'intrinsics_corners_nb': [11, 7], 'intrinsics_square_size': 30}}

extrinsics_config_dict = {'calculate_extrinsics':True, 'show_detection_extrinsics':False, 'extrinsics_method':'board',
                          'extrinsics_extension': 'png', 'extrinsics_corners_nb':[11,7], 'extrinsics_square_size':30,
                          'board': {'extrinsics_extension': 'png', 'show_reprojection_error': True,
                                    'extrinsics_corners_nb': [11, 7], 'extrinsics_square_size': 30}}

calib_dir = rcam
ret, C, S, D, K, R, T = calibration.calib_calc_fun(calib_dir, intrinsics_config_dict, extrinsics_config_dict)

print(f'reproj error (mm): {ret} \nCam: {C} \nImage Size: {S} \nDistorsions: {D} \nIntrinsics: {K} \nR: {R} \nT (m): {T}')


reproj error (mm): [1.8528506] 
 Cam: ['calibration'] 
 Image Size: [[1024.0, 576.0]] 
 Distorsions: [array([ 0.00171988, -0.05473393, -0.00141958,  0.00124846,  0.        ])] 
 Intrinsics: [array([[722.16141241,   0.        , 513.55680097],
       [  0.        , 732.82217137, 292.96790798],
       [  0.        ,   0.        ,   1.        ]])] 
R: [array([-0.0076784 , -0.02694066,  0.01285051])] 
T: [array([-0.10418057, -0.13736485,  0.53347749])]


In [9]:
calib_dir = lcam

ret, C, S, D, K, R, T = calibration.calib_calc_fun(calib_dir, intrinsics_config_dict, extrinsics_config_dict)

print(f'reproj error (mm): {ret} \nCam: {C} \nImage Size: {S} \nDistorsions: {D} \nIntrinsics: {K} \nR: {R} \nT (m): {T}')


reproj error (mm): [1.6600986] 
Cam: ['calibration'] 
Image Size: [[1024.0, 576.0]] 
Distorsions: [array([ 0.01308264, -0.06773213,  0.00083728,  0.00010534,  0.        ])] 
Intrinsics: [array([[718.20586072,   0.        , 524.24014514],
       [  0.        , 729.25951051, 285.53750339],
       [  0.        ,   0.        ,   1.        ]])] 
R: [array([-0.03799997,  0.00696002,  0.02125898])] 
T (m): [array([-0.27465367, -0.12081845,  0.54168781])]
